# Visualizing Debugging Tensors of MXNet training

### Overview

SageMaker Debugger is a new capability of Amazon SageMaker that allows debugging machine learning models. 
It lets you go beyond just looking at scalars like losses and accuracies during training and gives 
you full visibility into all the tensors 'flowing through the graph' during training. SageMaker Debugger helps you to monitor your training in near real time using rules and would provide you alerts, once it has detected an inconsistency in the training flow.

Using SageMaker Debugger is a two step process: Saving tensors and Analysis. In this notebook we will run an MXNet training job and configure SageMaker Debugger to store all tensors from this job. Afterwards we will visualize those tensors in our notebook.


### Dependencies
Before we begin, let us install the library plotly if it is not already present in the environment.
If the below cell installs the library for the first time, you'll have to restart the kernel and come back to the notebook. In addition to that, in order for our vizualiation to access tensors let's install smdebug - debugger library that provides API access to tensors emitted during training job.

In [8]:
! python -m pip install plotly
! python -m pip install smdebug

  Using cached smdebug-1.0.7-py2.py3-none-any.whl (263 kB)
  Using cached pyinstrument-3.4.1-py2.py3-none-any.whl (81 kB)
  Using cached pyinstrument_cext-0.2.4-cp36-cp36m-manylinux2010_x86_64.whl (20 kB)


### Configure and run the training job

Now we'll call the Sagemaker MXNet Estimator to kick off a training job with Debugger attached to it.

The `entry_point_script` points to the MXNet training script.

In [1]:
import os
import sagemaker
from sagemaker.debugger import DebuggerHookConfig, CollectionConfig
from sagemaker.mxnet import MXNet

sagemaker_session = sagemaker.Session()

entry_point_script = './scripts/mxnet_gluon_save_all_demo.py'
hyperparameters = {'batch-size': 256}
base_job_name = 'mnist-tensor-plot'

# Make sure to set this to your bucket and location
BUCKET_NAME = sagemaker_session.default_bucket()
LOCATION_IN_BUCKET = 'mnist-tensor-plot'
s3_bucket_for_tensors = 's3://{BUCKET_NAME}/{LOCATION_IN_BUCKET}'.format(BUCKET_NAME=BUCKET_NAME, LOCATION_IN_BUCKET=LOCATION_IN_BUCKET)

estimator = MXNet(role=sagemaker.get_execution_role(),
                  base_job_name=base_job_name,
                  train_instance_count=1,
                  train_instance_type='ml.m4.xlarge',
                  entry_point=entry_point_script,
                  framework_version='1.6.0',
                  train_max_run=3600,
                  sagemaker_session=sagemaker_session,
                  py_version='py3',
                  debugger_hook_config = DebuggerHookConfig(
                      s3_output_path=s3_bucket_for_tensors,  # Required
                      collection_configs=[
                          CollectionConfig(
                              name="all_tensors",
                              parameters={
                                  "include_regex": ".*",
                                  "save_steps": "1, 2, 3"
                              }
                          )
                      ]
                  ))

train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Estimator described above will save all tensors of all layers during steps 1, 2 and 3. Now, let's start the training job:

In [2]:
estimator.fit()

2021-03-28 13:14:24 Starting - Starting the training job...
2021-03-28 13:14:48 Starting - Launching requested ML instancesProfilerReport-1616937264: InProgress
......
2021-03-28 13:15:53 Starting - Preparing the instances for training.........
2021-03-28 13:17:21 Downloading - Downloading input data...
2021-03-28 13:17:48 Training - Downloading the training image.2021-03-28 13:17:58,622 sagemaker-training-toolkit INFO     Imported framework sagemaker_mxnet_container.training
2021-03-28 13:17:58,625 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-03-28 13:17:58,643 sagemaker_mxnet_container.training INFO     MXNet training environment: {'SM_HOSTS': '["algo-1"]', 'SM_NETWORK_INTERFACE_NAME': 'eth0', 'SM_HPS': '{}', 'SM_USER_ENTRY_POINT': 'mxnet_gluon_save_all_demo.py', 'SM_FRAMEWORK_PARAMS': '{}', 'SM_RESOURCE_CONFIG': '{"current_host":"algo-1","hosts":["algo-1"],"network_interface_name":"eth0"}', 'SM_INPUT_DATA_CONFIG': '{}', 'SM_OUTPUT_DATA_DIR'


2021-03-28 13:18:24 Uploading - Uploading generated training modelEpoch 0: loss 0.418, train acc 0.871, test acc 0.060, in 18.6 sec
2021-03-28 13:18:23,797 sagemaker-training-toolkit INFO     Reporting training SUCCESS

2021-03-28 13:18:49 Completed - Training job completed
Training seconds: 67
Billable seconds: 67


### Get S3 location of tensors

Now we can retrieve the S3 location of the tensors:

In [3]:
tensors_path = estimator.latest_job_debugger_artifacts_path()
print('S3 location of tensors is: ', tensors_path)

S3 location of tensors is:  s3://sagemaker-us-west-2-230755935769/mnist-tensor-plot/mnist-tensor-plot-2021-03-28-13-14-24-061/debug-output


In [9]:
import smdebug
from smdebug.trials import create_trial
trial = create_trial(tensors_path)

[2021-03-28 13:22:10.355 ip-172-16-51-26:19614 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-03-28 13:22:10.368 ip-172-16-51-26:19614 INFO s3_trial.py:42] Loading trial debug-output at path s3://sagemaker-us-west-2-230755935769/mnist-tensor-plot/mnist-tensor-plot-2021-03-28-13-14-24-061/debug-output


In [10]:
trial.tensor_names()

[2021-03-28 13:22:15.936 ip-172-16-51-26:19614 INFO trial.py:198] Training has ended, will refresh one final time in 1 sec.
[2021-03-28 13:22:16.957 ip-172-16-51-26:19614 INFO trial.py:210] Loaded all steps


['conv0_bias',
 'conv0_input_0',
 'conv0_output_0',
 'conv0_relu_input_0',
 'conv0_relu_output_0',
 'conv0_weight',
 'conv1_bias',
 'conv1_input_0',
 'conv1_output_0',
 'conv1_relu_input_0',
 'conv1_relu_output_0',
 'conv1_weight',
 'dense0_bias',
 'dense0_input_0',
 'dense0_output_0',
 'dense0_relu_input_0',
 'dense0_relu_output_0',
 'dense0_weight',
 'dense1_bias',
 'dense1_input_0',
 'dense1_output_0',
 'dense1_relu_input_0',
 'dense1_relu_output_0',
 'dense1_weight',
 'dense2_bias',
 'dense2_input_0',
 'dense2_output_0',
 'dense2_weight',
 'flatten0_input_0',
 'flatten0_output_0',
 'gradient/conv0_bias',
 'gradient/conv0_weight',
 'gradient/conv1_bias',
 'gradient/conv1_weight',
 'gradient/dense0_bias',
 'gradient/dense0_weight',
 'gradient/dense1_bias',
 'gradient/dense1_weight',
 'gradient/dense2_bias',
 'gradient/dense2_weight',
 'hybridsequential0_input_0',
 'hybridsequential0_output_0',
 'pool0_input_0',
 'pool0_output_0',
 'pool1_input_0',
 'pool1_output_0',
 'softmaxcrossent

### Download tensors from S3

Next we download the tensors from S3, so that we can visualize them in the notebook.

In [11]:
folder_name = tensors_path.split("/")[-1]
os.system("aws s3 cp --recursive " + tensors_path + " " + folder_name)
print('Downloaded tensors into folder: ', folder_name)

Downloaded tensors into folder:  debug-output


### Visualize
The main purpose of this class (TensorPlot) is to visualise the tensors in your network. This could be to determine dead or saturated activations, or the features maps the network.

To use this class (TensorPlot), you will need to supply the argument regex with the tensors you are interested in. e.g., if you are interested in activation outputs, then you need to supply the following regex .*relu|.*tanh|.*sigmoid.

Another important argument is the `sample_batch_id`, which allows you to specify the index of the batch size to display. For example, given an input tensor of size (batch_size, channel, width, height), `sample_batch_id = n` will display (n, channel, width, height). If you set sample_batch_id = -1 then the tensors will be summed over the batch dimension (i.e., `np.sum(tensor, axis=0)`). If batch_sample_id is None then each sample will be plotted as separate layer in the figure.

Here are some interesting use cases:

1) If you want to determine dead or saturated activations for instance ReLus that are always outputting zero, then you would want to sum the batch dimension (sample_batch_id=-1). The sum gives an indication which parts of the network are inactive across a batch.

2) If you are interested in the feature maps for the first image in the batch, then you should provide batch_sample_id=0. This can be helpful if your model is not performing well for certain set of samples and you want to understand which activations are leading to misprediction.

An example visualization of layer outputs:
![](./images/tensorplot.gif)


`TensorPlot` normalizes tensor values to the range 0 to 1 which means colorscales are the same across layers. Blue indicates value close to 0 and yellow indicates values close to 1. This class has been designed to plot convolutional networks that take 2D images as input and predict classes or produce output images. You can use this  for other types of networks like RNNs, but you may have to adjust the class as it is currently neglecting tensors that have more than 4 dimensions.

Let's plot Relu output activations for the given MNIST training example.

In [12]:
import tensor_plot 

visualization = tensor_plot.TensorPlot(
#     regex=".*relu_output", 
      regex=".*weight", 
    path=folder_name,
    steps=10,  
    batch_sample_id=0,
    color_channel = 1,
    title="Relu outputs",
    label=".*sequential0_input_0",
    prediction=".*sequential0_output_0"
)

[2021-03-28 13:24:40.353 ip-172-16-51-26:19614 INFO local_trial.py:35] Loading trial debug-output at path debug-output
[2021-03-28 13:24:40.358 ip-172-16-51-26:19614 INFO trial.py:198] Training has ended, will refresh one final time in 1 sec.
[2021-03-28 13:24:41.360 ip-172-16-51-26:19614 INFO trial.py:210] Loaded all steps


If we plot too many layers, it can crash the notebook. If you encounter performance or out of memory issues, then either try to reduce the layers to plot by changing the `regex` or run this Notebook in JupyterLab instead of Jupyter. 

In the below cell we vizualize outputs of all layers, including final classification. Please note that because training job ran only for 1 epoch classification accuracy is not high.

In [13]:
visualization.fig.show(renderer="iframe")

For additional example of working with debugging tensors and visualizing them in real time please feel free to try it out at [MXNet realtime analysis](../mxnet_realtime_analysis/mxnet-realtime-analysis.ipynb) example.